In [5]:
# Install TensorFlow (if not already installed)
!pip install tensorflow

import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


In [6]:
# Load the dataset
file_path = '/content/df_cleaned_v2.csv'  # Adjust the path as necessary after uploading your file
df = pd.read_csv(file_path)

In [7]:
# Drop rows with missing values
df = df.dropna()

In [8]:
# Encode categorical features
categorical_features = ['brand', 'Kondisi', 'Dirakit', 'InjeksiLangsung', 'Aspirasi']
label_encoders = {}

for col in categorical_features:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [9]:
# Split the data into features and targets
X = df.drop(columns=['price'])
y = df['price']

In [10]:
# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [11]:
# Split the data into training and testing sets for both classification and regression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Create a categorical target for classification by binning the price
y_classification = pd.qcut(y, q=3, labels=False)  # Bin the prices into 3 categories
y_train_cls, y_test_cls = train_test_split(y_classification, test_size=0.2, random_state=42)


In [13]:
# Check shapes
(X_train.shape, X_test.shape, y_train.shape, y_test.shape, y_train_cls.shape, y_test_cls.shape)


((10174, 14), (2544, 14), (10174,), (2544,), (10174,), (2544,))

In [14]:
# Define the classification model
classification_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_dim=X_train.shape[1], activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 output classes
])

classification_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
# Train the classification model
classification_model.fit(X_train, y_train_cls, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
255/255 [==============================] - 2s 3ms/step - loss: 0.5201 - accuracy: 0.7854 - val_loss: 0.4176 - val_accuracy: 0.8236
Epoch 2/50
255/255 [==============================] - 1s 2ms/step - loss: 0.3647 - accuracy: 0.8468 - val_loss: 0.3602 - val_accuracy: 0.8585
Epoch 3/50
255/255 [==============================] - 1s 3ms/step - loss: 0.3221 - accuracy: 0.8673 - val_loss: 0.3274 - val_accuracy: 0.8737
Epoch 4/50
255/255 [==============================] - 1s 2ms/step - loss: 0.2975 - accuracy: 0.8754 - val_loss: 0.3144 - val_accuracy: 0.8776
Epoch 5/50
255/255 [==============================] - 1s 2ms/step - loss: 0.2758 - accuracy: 0.8855 - val_loss: 0.2955 - val_accuracy: 0.8830
Epoch 6/50
255/255 [==============================] - 1s 2ms/step - loss: 0.2594 - accuracy: 0.8919 - val_loss: 0.2812 - val_accuracy: 0.8870
Epoch 7/50
255/255 [==============================] - 0s 2ms/step - loss: 0.2465 - accuracy: 0.8973 - val_loss: 0.2660 - val_accuracy: 0.8929
Epoch 

In [16]:
# Evaluate the classification model
classification_loss, classification_accuracy = classification_model.evaluate(X_test, y_test_cls)
print(f"Classification Accuracy: {classification_accuracy}")

80/80 [==============================] - 0s 1ms/step - loss: 0.1532 - accuracy: 0.9375
Classification Accuracy: 0.9375


In [17]:
# Define the regression model
regression_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_dim=X_train.shape[1], activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Single output for regression
])

regression_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [18]:
# Train the regression model
regression_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
255/255 [==============================] - 2s 3ms/step - loss: 279911436816220160.0000 - mae: 429867808.0000 - val_loss: 293748103457013760.0000 - val_mae: 442962496.0000
Epoch 2/50
255/255 [==============================] - 1s 3ms/step - loss: 279909701649432576.0000 - mae: 429866400.0000 - val_loss: 293744152087101440.0000 - val_mae: 442959520.0000
Epoch 3/50
255/255 [==============================] - 1s 4ms/step - loss: 279903156119273472.0000 - mae: 429861408.0000 - val_loss: 293733741086375936.0000 - val_mae: 442951776.0000
Epoch 4/50
255/255 [==============================] - 1s 3ms/step - loss: 279889137346019328.0000 - mae: 429850848.0000 - val_loss: 293713606279692288.0000 - val_mae: 442936928.0000
Epoch 5/50
255/255 [==============================] - 1s 4ms/step - loss: 279865222968115200.0000 - mae: 429833152.0000 - val_loss: 293682407637254144.0000 - val_mae: 442914016.0000
Epoch 6/50
255/255 [==============================] - 1s 4ms/step - loss: 2798301073155031

In [19]:
# Evaluate the regression model
regression_loss, regression_mae = regression_model.evaluate(X_test, y_test)
print(f"Regression MAE: {regression_mae}")

80/80 [==============================] - 0s 1ms/step - loss: 260719925209858048.0000 - mae: 414486816.0000
Regression MAE: 414486816.0
